In [2]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType

schema = StructType(
      [
        StructField("name", StringType(), False),
        StructField("price", DoubleType(), False),
        StructField("timestamp", TimestampType(), False),
      ]
    )

In [4]:
kafka_server = "kafka1:9092"   
from pyspark.sql.functions import from_json

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "stock")                       # Subscribe to the "en" Kafka topic 
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
# Load the DataFrame
)
df = lines.select("parsed_value.*")


## The assignment starts here

You can create a


## Select the stock that gained the most (between windows)

In [8]:
#remember you can register another stream

from pyspark.sql.functions import lag, window

# Hàm xử lý từng batch dữ liệu
def process_batch(df, epoch_id):
    
    window_spec = Window.partitionBy("name").orderBy("window")

    # Tạo một cột "previous_timestamp" bằng cách sử dụng hàm lag
    df = df.withColumn("previous_price", lag("avg(price)").over(window_spec))
    df = df.withColumn("previous_window", lag("window").over(window_spec))
    df = df.filter(df["previous_price"] < df["avg(price)"])
    
    # Tiến hành xử lý dữ liệu dưới đây, ví dụ: in ra màn hình
    df.show(truncate=False)
    
    
windowedDF_3 = df \
        .withWatermark("timestamp", "30 seconds") \
        .groupBy(window("timestamp", "5 minutes"), "name") \
        .agg({"price": "avg"})

gained_value_stocks = windowedDF_3.orderBy("avg(price)", ascending=False)


# In kết quả
query= (gained_value_stocks.writeStream
           .outputMode("complete")
           .format("memory")
           .queryName("GainedValueStocks1")
           .option("truncate", False)
           .foreachBatch(process_batch)
           .start())
    
query.awaitTermination()

23/11/06 10:42:27 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bead8334-f1a7-41f5-8836-f322647bed47. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


+------------------------------------------+----+------------------+--------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price|previous_window                           |
+------------------------------------------+----+------------------+--------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02        |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66        |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99        |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 11:20:00, 2023-11-05 11:25:00]|GIS |50.92             |48.6          |[2023-11-05 11:15:00, 2023-11-05 11:20:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|GIS |57.61             |50.92         |[2023-11-05 11:20:00, 2023-1

+------+----+----------+--------------+---------------+
|window|name|avg(price)|previous_price|previous_window|
+------+----+----------+--------------+---------------+
+------+----+----------+--------------+---------------+



+------------------------------------------+----+------------------+--------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price|previous_window                           |
+------------------------------------------+----+------------------+--------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02        |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66        |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99        |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 11:20:00, 2023-11-05 11:25:00]|GIS |50.92             |48.6          |[2023-11-05 11:15:00, 2023-11-05 11:20:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|GIS |57.61             |50.92         |[2023-11-05 11:20:00, 2023-1

+------+----+----------+--------------+---------------+
|window|name|avg(price)|previous_price|previous_window|
+------+----+----------+--------------+---------------+
+------+----+----------+--------------+---------------+

+------------------------------------------+----+------------------+--------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price|previous_window                           |
+------------------------------------------+----+------------------+--------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02        |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66        |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99        |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 

...

+------------------------------------------+----+------------------+------------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price    |previous_window                           |
+------------------------------------------+----+------------------+------------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02            |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66            |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99            |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 11:50:00, 2023-11-05 11:55:00]|ALXN|139.99            |127.81            |[2023-11-05 11:45:00, 2023-11-05 11:50:00]|
|[2023-11-05 12:00:00, 2023-11-05 12:05:00]|ALXN|116.72            |104.12333333333

+------------------------------------------+----+------------------+--------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price|previous_window                           |
+------------------------------------------+----+------------------+--------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02        |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66        |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99        |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 11:50:00, 2023-11-05 11:55:00]|ALXN|139.99            |127.81        |[2023-11-05 11:45:00, 2023-11-05 11:50:00]|
|[2023-11-05 11:20:00, 2023-11-05 11:25:00]|GIS |50.92             |48.6          |[2023-11-05 11:15:00, 2023-1

+------------------------------------------+----+------------------+------------------+------------------------------------------+
|window                                    |name|avg(price)        |previous_price    |previous_window                           |
+------------------------------------------+----+------------------+------------------+------------------------------------------+
|[2023-11-05 11:15:00, 2023-11-05 11:20:00]|ALXN|180.2             |108.02            |[2023-11-05 11:10:00, 2023-11-05 11:15:00]|
|[2023-11-05 11:25:00, 2023-11-05 11:30:00]|ALXN|186.99            |164.66            |[2023-11-05 11:20:00, 2023-11-05 11:25:00]|
|[2023-11-05 11:30:00, 2023-11-05 11:35:00]|ALXN|201.66995         |186.99            |[2023-11-05 11:25:00, 2023-11-05 11:30:00]|
|[2023-11-05 11:50:00, 2023-11-05 11:55:00]|ALXN|139.99            |127.81            |[2023-11-05 11:45:00, 2023-11-05 11:50:00]|
|[2023-11-05 12:00:00, 2023-11-05 12:05:00]|ALXN|116.72            |104.12333333333

ERROR:root:Exception while sending command.age 1821:>             (0 + 0) / 200]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty

Py4JError: An error occurred while calling o949.awaitTermination